In [1]:
print("hello world")

hello world


In [2]:
%pip install deepface

  Preparing metadata (setup.py) ... done
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.6-py3-none-any.whl.metadata (4.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 24.5 MB/s eta 0:00:00
Using cached click-8.1.7-py3-none-any.whl (97 kB)
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
Using cached soupsieve-2.6-py3-none-any.whl (36 kB)
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=c081ac68978b9013a289bed6ae320835d539713ee041b5061bebe58f2f035fca
  Stored in directory: /Users/rgopalam/Library/Caches/pip/wheels/3b/ee/ac/319a7b7f331f61050d0d54425079b2a883b445be3c7284a4eb
Successfully built fire
Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests

url = "https://storage.googleapis.com/mediapipe-models/face_detector/blaze_face_short_range/float16/1/blaze_face_short_range.tflite"
output_path = "detector.tflite"

response = requests.get(url)
if response.status_code == 200:
    with open(output_path, "wb") as f:
        f.write(response.content)
    print(f"Downloaded to {output_path}")
else:
    print(f"Failed to download. HTTP Status Code: {response.status_code}")



Downloaded to detector.tflite


In [4]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2

detection_results = []  # Global variable to store results for annotation
def print_result(result, output_image, timestamp_ms):
    global detection_results
    detection_results = result.detections  # Store detections for annotation

# Visualization function
def visualize(image, detections):
    """Draw bounding boxes on the image."""
    annotated_image = image.copy()
    height, width, _ = image.shape
    for detection in detections:
        bbox = detection.bounding_box
        start_point = (bbox.origin_x, bbox.origin_y)
        end_point = (bbox.origin_x + bbox.width, bbox.origin_y + bbox.height)
        # Draw the bounding box
        cv2.rectangle(annotated_image, start_point, end_point, (0, 255, 0), 2)
    return annotated_image



# Load the MediaPipe model
BaseOptions = mp.tasks.BaseOptions
FaceDetector = mp.tasks.vision.FaceDetector
FaceDetectorOptions = mp.tasks.vision.FaceDetectorOptions
VisionRunningMode = mp.tasks.vision.RunningMode

# Path to the downloaded model
model_path = "detector.tflite"

# Set options for live stream
options = FaceDetectorOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.LIVE_STREAM,
    result_callback=print_result
)



2024-12-11 02:28:14.300944: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
#from fer import FER

In [ ]:
import cv2
import mediapipe as mp
from deepface import DeepFace

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)


# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils



# Initialize FER
#emotion_detector = FER(mtcnn=True)

# Access the webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Unable to access the camera.")
else:
    print("Camera is on. Press 'q' to quit.")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Unable to read frame.")
            break

        # Convert the frame to RGB for MediaPipe and FER
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect face mesh
        results_face = face_mesh.process(rgb_frame)
        if results_face.multi_face_landmarks:
            for face_landmarks in results_face.multi_face_landmarks:
                # Draw the facial landmarks on the frame
                mp.solutions.drawing_utils.draw_landmarks(
                    frame, face_landmarks, mp_face_mesh.FACEMESH_TESSELATION,
                    mp.solutions.drawing_styles.get_default_face_mesh_tesselation_style())


        try:
            result = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
            # Display dominant emotion on the frame
            dominant_emotion = max(result['emotion'], key=result['emotion'].get)
            cv2.putText(frame, dominant_emotion, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        except Exception as e:
            print("No face detected:", str(e))
    

        # if results_face.multi_face_landmarks:
        #     for face_landmarks in results_face.multi_face_landmarks:
        #         # Example: Detect smile
        #         landmarks = face_landmarks.landmark
        #         # Calculate distances
        #         left_corner = landmarks[61]  # Left corner of mouth
        #         right_corner = landmarks[291]  # Right corner of mouth
        #         upper_lip = landmarks[13]  # Upper lip
        #         lower_lip = landmarks[14]  # Lower lip

        #         # Horizontal distance (mouth width)
        #         mouth_width = ((right_corner.x - left_corner.x) ** 2 +
        #                     (right_corner.y - left_corner.y) ** 2) ** 0.5
        #         # Vertical distance (mouth openness)
        #         mouth_height = ((upper_lip.x - lower_lip.x) ** 2 +
        #                         (upper_lip.y - lower_lip.y) ** 2) ** 0.5

        #         # Define a smile condition
        #         if mouth_width / mouth_height > 3:  # Adjust the threshold as needed
        #             cv2.putText(frame, "Smiling", (50, 50),
        #                         cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Process the frame to detect pose
        results_pose = pose.process(rgb_frame)

        # Draw landmarks on the frame
        if results_pose.pose_landmarks:
            mp_drawing.draw_landmarks(
                frame, results_pose.pose_landmarks, mp_pose.POSE_CONNECTIONS
            )

        # Display the frame
        cv2.imshow("MediaPipe Pose", frame)



        # Detect emotions
        # emotions = emotion_detector.detect_emotions(frame)
        # if emotions:
        #     dominant_emotion, emotion_probabilities = emotions[0]['emotions'].items(), emotions[0]['emotions']
        #     emotion_text = f"Emotion: {max(emotion_probabilities, key=emotion_probabilities.get)}"
        #     cv2.putText(frame, emotion_text, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        # Display the frame
        cv2.imshow('Face Mesh & Emotion Detection', frame)

        # Exit when 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Turning off the camera.")
            break

# Release resources
cap.release()
cv2.destroyAllWindows()


24-12-11 02:30:39 - Directory /Users/rgopalam/.deepface has been created
24-12-11 02:30:39 - Directory /Users/rgopalam/.deepface/weights has been created


I0000 00:00:1733913040.072733 2756354 gl_context.cc:357] GL version: 2.1 (2.1 ATI-4.14.1), renderer: AMD Radeon Pro 555 OpenGL Engine
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1733913040.079504 2764936 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733913040.092054 2764936 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1733913040.093323 2756354 gl_context.cc:357] GL version: 2.1 (2.1 ATI-4.14.1), renderer: AMD Radeon Pro 555 OpenGL Engine
W0000 00:00:1733913040.573138 2764951 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1733913040.623356 2764954 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature

Camera is on. Press 'q' to quit.
No face detected: module 'deepface.modules.modeling' has no attribute 'build_model'


W0000 00:00:1733913041.636121 2764943 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


No face detected: module 'deepface.modules.modeling' has no attribute 'build_model'
No face detected: module 'deepface.modules.modeling' has no attribute 'build_model'
No face detected: module 'deepface.modules.modeling' has no attribute 'build_model'
No face detected: module 'deepface.modules.modeling' has no attribute 'build_model'
No face detected: module 'deepface.modules.modeling' has no attribute 'build_model'
No face detected: module 'deepface.modules.modeling' has no attribute 'build_model'
No face detected: module 'deepface.modules.modeling' has no attribute 'build_model'
No face detected: module 'deepface.modules.modeling' has no attribute 'build_model'
No face detected: module 'deepface.modules.modeling' has no attribute 'build_model'
No face detected: module 'deepface.modules.modeling' has no attribute 'build_model'
No face detected: module 'deepface.modules.modeling' has no attribute 'build_model'
No face detected: module 'deepface.modules.modeling' has no attribute 'build

: 

In [7]:
%pip install tf-keras

INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.0 MB/s eta 0:00:00ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2

# Access the webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Unable to access the camera.")
    exit()

# Initialize a manual timestamp counter
current_timestamp_ms = 0
timestamp_increment = 33  # Adjust for your camera's frame rate

# Create the face detector instance
with FaceDetector.create_from_options(options) as detector:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame.")
            break

        # Resize frame to improve performance
        frame = cv2.resize(frame, (640, 480))

        # Convert frame to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Convert to MediaPipe Image
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_rgb)

        try:
            # Increment and send manual timestamp
            current_timestamp_ms += timestamp_increment
            detector.detect_async(mp_image, current_timestamp_ms)
        except ValueError as e:
            print(f"Error in detection: {e}")
            break


        # Draw bounding boxes on the frame using the latest results
        annotated_frame = visualize(frame, detection_results)

        # Display the annotated frame
        cv2.imshow("Face Detection", annotated_frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):  # Exit on 'q' key press
            break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1733909744.847078 2698198 gl_context.cc:357] GL version: 2.1 (2.1 ATI-4.14.1), renderer: AMD Radeon Pro 555 OpenGL Engine
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1733909744.874495 2698901 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


: 